In [ ]:
import pandas as pd
df = pd.read_csv('../data/hansard_speeches_processed.csv')
df = df.sample(n=50000, random_state = 42)

import sys
sys.path.append('../gtm/')
from corpus import GTMCorpus
from gtm import GTM

# Create a GTMCorpus object
train_dataset = GTMCorpus(
    df, 
    embeddings_type = None,
    prevalence = "~ party", 
    content = "~ party"
)

# Train the model
tm = GTM(
    train_dataset, 
    doc_topic_prior='logistic_normal',
    update_prior=True,
    num_epochs=10
)

In [ ]:
# Assess the quality of the learned word embeddings 
# Top 8 closest words to a specific word

import torch
import torch.nn.functional as F

specific_word = 'crime'

word_id = [i for i,w in enumerate(train_dataset.vocab) if w == specific_word][0]

words = tm.AutoEncoder.decoder['dec_1'].weight.T

logit = torch.matmul(words.T[word_id], words)

beta = F.softmax(logit)

tm.AutoEncoder.eval()
topic_words = []
vals, indices = torch.topk(beta, 8)
vals = vals.cpu().tolist()
indices = indices.cpu().tolist()
[tm.id2token[idx] for idx in indices]

In [ ]:
# Visualize the learned embeddings space 

# Topic, covariates, and word embeddings
topics_and_covs = tm.AutoEncoder.decoder['dec_0'].weight.T
words = tm.AutoEncoder.decoder['dec_1'].weight.T

# Dimension reduction (PCA 50 components + UMAP 2 components)
all_vecs = torch.cat([words.T, topics_and_covs], dim=0)
all_vecs = all_vecs.cpu().detach().numpy()

from sklearn.decomposition import PCA
pca_args={"n_components": 50, "svd_solver": "full"}
pca_model = PCA(**pca_args).fit(all_vecs)
all_vecs = pca_model.transform(all_vecs)

import umap
umap_args={"n_neighbors": 15, "n_components": 2, "random_state": 0}
umap_model = umap.umap_.UMAP(**umap_args).fit(all_vecs)

# Plot resulting vectors
import matplotlib.pyplot as plt
plt.figure(dpi=80)

words = words.T.cpu().detach().numpy()
words = pca_model.transform(words)
words = umap_model.transform(words)
plt.scatter(
    words[:,0],
    words[:,1],
    color=(0.5, 0.5, 1),
    alpha=0.5,
)

topics = topics_and_covs.cpu().detach().numpy()
topics = pca_model.transform(topics)
topics = umap_model.transform(topics)
plt.scatter(
    topics[:,0],
    topics[:,1],
    color=(0, 0.5, 0.5),
    alpha=1,
)

In [ ]:
# tm.get_topic_correlations()